# Imports

## Libraries

In [1]:
# Standard Library Imports
import datetime
import io
import re
import zipfile

# Third-Party Imports
import numpy as np
import pandas as pd
import requests
from scipy.stats import mode
import xgboost as xgb

from skrub import TableVectorizer

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    HistGradientBoostingRegressor,
)
from sklearn.pipeline import Pipeline

# from sklearn.preprocessing import (
#     FunctionTransformer,
#     OneHotEncoder,
#     OrdinalEncoder,
#     StandardScaler,
# )

## Data

### Provided data

In [2]:
# Import provided data
train = pd.read_parquet("/Users/pierrehaas/bike_counters/data/train.parquet")
test = pd.read_parquet("/Users/pierrehaas/bike_counters/data/final_test.parquet")

### External data

#### Weather data

In [3]:
# Import additionally sourced data

# https://meteo.data.gouv.fr/datasets/donnees-climatologiques-de-base-horaires/
weather = pd.read_csv(
    "/Users/pierrehaas/bike_counters/external_data/weather/H_75_previous-2020-2022.csv.gz",
    parse_dates=["AAAAMMJJHH"],
    date_format="%Y%m%d%H",
    compression="gzip",
    sep=";",
).rename(columns={"AAAAMMJJHH": "date"})

#### Public transport data

In [4]:
# https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/information/
# URLs of the zip files
urls = [
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/files/e6bcf4c994951fc086e31db6819a3448/download/",
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-ferre/files/e35b9ec0a183a8f2c7a8537dd43b124c/download/",
]

# Initialize an empty list to store DataFrames
dfs = []

# File matching pattern
pattern = r"data-rf-202\d/202\d_S\d+_NB_FER\.txt"

# Process each ZIP file
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Get a list of all files in the archive and filter matching files
            matching_files = [f for f in z.namelist() if re.match(pattern, f)]

            # Read and concatenate the matching files
            for file in matching_files:
                with z.open(file) as f:
                    # Assuming the files are tab-separated and have a "JOUR" column
                    df = pd.read_csv(f, sep="\t", parse_dates=["JOUR"], dayfirst=True)
                    dfs.append(df)

# Combine all DataFrames
underground_transport = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()


# # https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/information/
# # URLs of the zip files
urls = [
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/files/41adcbd4216382c232ced4ccbf60187e/download/",
    "https://data.iledefrance-mobilites.fr/explore/dataset/histo-validations-reseau-surface/files/68cac32e8717f476905a60006a4dca26/download/",
]

# Initialize an empty list to store DataFrames
dfs = []

# File matching pattern
pattern = r"data-rs-202\d/202\d_T\d+_NB_SURFACE\.txt"

# Process each ZIP file
for url in urls:
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Get a list of all files in the archive and filter matching files
            matching_files = [f for f in z.namelist() if re.match(pattern, f)]

            # Read and concatenate the matching files
            for file in matching_files:
                with z.open(file) as f:
                    # Assuming the files are tab-separated and have a "JOUR" column
                    df = pd.read_csv(
                        f,
                        sep="\t",
                        parse_dates=["JOUR"],
                        dayfirst=True,
                        encoding="latin1",
                    )
                    dfs.append(df)

# Combine all DataFrames
overground_transport = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

#### Car traffic data

In [5]:
# URLs of the zip files
urls = [
    "https://parisdata.opendatasoft.com/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2020_zip/",
    # "https://parisdata.opendatasoft.com/api/datasets/1.0/comptages-routiers-permanents-historique/attachments/opendata_txt_2021_zip/", # This file's compression format is broken, thus I provide a download link below
    "https://www.dropbox.com/scl/fi/sfqzlzpyxcf4yied3yucc/comptage-routier-2021.zip?rlkey=6k6hr3kywl8tvm4ax1qv2nv88&st=ktehiium&dl=1",
]

# Initialize an empty list to store DataFrames
dfs = []

# Process each ZIP file
for i, url in enumerate(urls):
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Process every file in the archive
            for file in z.namelist():
                # Skip directories and __MACOSX files
                if file.endswith("/") or "__MACOSX" in file:
                    continue
                # For the second URL, ensure files are within "comptage-routier-2021" directory
                if i == 1 and not file.startswith("comptage-routier-2021/"):
                    continue
                with z.open(file) as f:
                    # Assuming the files are semicolon-separated and have a "t_1h" column
                    df = pd.read_csv(f, sep=";", parse_dates=["t_1h"])
                    dfs.append(df)

# Combine all DataFrames
cars_count = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Feature engineering

## Date encoding

In [21]:
def date_encoder(X, col="date"):
    X = X.copy()  # modify a copy of X

    # Encode the date information from the DateOfDeparture columns
    X["year"] = X[col].dt.year
    X["quarter"] = X[col].dt.quarter
    X["month"] = X[col].dt.month
    X["day"] = X[col].dt.day
    X["weekday"] = X[col].dt.weekday + 1
    X["hour"] = X[col].dt.hour
    X["hour_scaled"] = np.cos(2 * np.pi * X["hour"] / 24)

    # Binary variable indicating weekend or not (1=weekend, 0=weekday)
    X["is_weekend"] = (X["weekday"] > 5).astype(int)

    # Binary variable indicating bank holiday or not (1=holiday, 0=not holiday)
    import holidays

    fr_bank_holidays = holidays.FR()  # Get list of FR holidays
    X["is_bank_holiday"] = X[col].apply(lambda x: 1 if x in fr_bank_holidays else 0)

    X = X.copy()  # modify a copy of X

    # Binary variable indicating school holiday or not (1=holiday, 0=not holiday)
    # https://www.data.gouv.fr/fr/datasets/vacances-scolaires-par-zones/
    fr_school_holidays = pd.read_csv(
        "/Users/pierrehaas/bike_counters/external_data/vacances_scolaires_france.csv"
    )[["date", "vacances_zone_c"]]

    # Ensure both DataFrames have a consistent datetime format
    X["date_normalized"] = pd.to_datetime(X[col]).dt.normalize()
    fr_school_holidays["date"] = pd.to_datetime(
        fr_school_holidays["date"]
    ).dt.normalize()

    # Create a dictionary from the holidays dataset for faster lookup
    holiday_mapping = dict(
        zip(fr_school_holidays["date"], fr_school_holidays["vacances_zone_c"])
    )

    # Map the normalized date to the holiday column
    X["is_school_holiday"] = (
        X["date_normalized"].map(holiday_mapping).fillna(0).astype(int)
    )

    # Drop the normalized date column if not needed
    X.drop(columns=["date_normalized"], inplace=True)

    # Finally, return the updated DataFrame
    return X

## Feature engineering on train and test data

In [22]:
def train_test_processing(X_train, X_test):

    def dist_station(X):

        lat_long_station = (
            X[["latitude", "longitude"]]
            - mode(
                X[X["site_name"] == "Totem 73 boulevard de Sébastopol"][
                    ["latitude", "longitude"]
                ]
            )[0]
        )

        # We calculate the distance the previously identified site to all others
        # We call the result the distance to the center since the station is close to the center of Paris
        dist_station = np.linalg.norm(
            lat_long_station,
            axis=1,
        )

        return dist_station

    X_train["dist_to_station"] = dist_station(X_train)
    X_test["dist_to_station"] = dist_station(X_test)

    # Group by counter_name and calculate the mean log_bike_count
    grouped_train = (
        X_train.groupby("counter_name", observed=True)["log_bike_count"]
        .mean()
        .reset_index()
    )

    # Reshape the data for clustering
    Y = grouped_train[["log_bike_count"]]

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=5)
    grouped_train["cluster"] = kmeans.fit_predict(Y)

    # Sort clusters by mean log_bike_count and reassign cluster labels
    sorted_clusters = (
        grouped_train.groupby("cluster")["log_bike_count"].mean().sort_values().index
    )
    cluster_mapping = {
        old_label: new_label for new_label, old_label in enumerate(sorted_clusters)
    }
    grouped_train["cluster"] = grouped_train["cluster"].map(cluster_mapping)

    # Merge the cluster labels back to the original DataFrame
    X_train = X_train.merge(
        grouped_train[["counter_name", "cluster"]], on="counter_name", how="left"
    )
    X_test = X_test.merge(
        grouped_train[["counter_name", "cluster"]], on="counter_name", how="left"
    )

    return X_train, X_test

## Feature engineering on weather data

In [23]:
def weather_processing(X, train_min, train_max, test_min, test_max):

    X_reduced = (
        X.drop(columns=["NUM_POSTE", "NOM_USUEL", "LAT", "LON", "QDXI3S"])
        .groupby("date")
        .mean()
        .dropna(axis=1, how="all")
        .interpolate(method="linear")
    )

    X_reduced["is_rain"] = (X_reduced["RR1"] > 0).astype(int)

    X_reduced["q_rain_lag_1h"] = X_reduced["RR1"].shift(1)
    X_reduced["t_rain_lag_1h"] = X_reduced["DRR1"].shift(1)
    X_reduced["is_rain_lag_1h"] = X_reduced["is_rain"].shift(1)

    X_reduced["q_rain_next_1h"] = X_reduced["RR1"].shift(-1)
    X_reduced["t_rain_next_1h"] = X_reduced["DRR1"].shift(-1)
    X_reduced["is_rain_next_1h"] = X_reduced["is_rain"].shift(-1)

    X_reduced["temp_lag_1h"] = X_reduced["T"].shift(1)
    X_reduced["temp_next_1h"] = X_reduced["T"].shift(-1)

    X_reduced["max_temp"] = X_reduced.groupby(X_reduced.index.date)["T"].transform(
        "max"
    )
    X_reduced["will_rain"] = (
        X_reduced.groupby(X_reduced.index.date)["RR1"]
        .transform(lambda x: (x > 0).any())
        .astype(int)
    )

    weather_features = [
        "RR1",
        "DRR1",
        "T",
        "TNSOL",
        "TCHAUSSEE",
        "U",
        "GLO",
        # "is_rain",
        "q_rain_lag_1h",
        "t_rain_lag_1h",
        # "is_rain_lag_1h",
        "q_rain_next_1h",
        "t_rain_next_1h",
        # "is_rain_next_1h",
        "temp_lag_1h",
        "temp_next_1h",
        "max_temp",
        # "will_rain",
    ]

    X_reduced_train = X_reduced[
        (X_reduced.index >= train_min) & (X_reduced.index <= train_max)
    ]

    X_reduced_test = X_reduced[
        (X_reduced.index >= test_min) & (X_reduced.index <= test_max)
    ]

    n = 5
    pca = PCA(n_components=n)

    pca.fit(X_reduced_train[weather_features])

    X_pca_train = pca.transform(X_reduced_train[weather_features])
    X_pca_test = pca.transform(X_reduced_test[weather_features])

    X_pca_train = pd.DataFrame(
        X_pca_train,
        index=X_reduced_train[weather_features].index,
        columns=["weather_" + str(i) for i in range(1, n + 1)],
    ).reset_index()

    X_pca_test = pd.DataFrame(
        X_pca_test,
        index=X_reduced_test[weather_features].index,
        columns=["weather_" + str(i) for i in range(1, n + 1)],
    ).reset_index()

    X_pca = pd.concat([X_pca_train, X_pca_test], ignore_index=True)

    X_pca = X_pca.merge(
        X_reduced[(X_reduced.index >= train_min) & (X_reduced.index <= test_max)][
            ["is_rain", "is_rain_lag_1h", "is_rain_next_1h", "will_rain"]
        ],
        on="date",
    )

    return X_pca

## Processing of public transport data

In [24]:
def transport_processing(X1, X2, train_min="2020-09-01", test_max="2021-10-18"):

    daily_X1 = X1.groupby("JOUR")["NB_VALD"].sum()
    daily_X2 = X2.groupby("JOUR")["NB_VALD"].sum()

    X = (daily_X1 + daily_X2).reset_index()

    X_reduced = date_encoder(X, col="JOUR")[
        (X["JOUR"] >= train_min) & (X["JOUR"] <= test_max)
    ]

    return X_reduced

## Processing of car traffic data

In [25]:
def car_traffic_processing(X, train_min, test_max):
    X_hourly = (
        X[
            (X["t_1h"] >= train_min - datetime.timedelta(hours=1))
            & (X["t_1h"] <= test_max + datetime.timedelta(hours=1))
        ]
        .groupby("t_1h")["q"]
        .sum()
        .reset_index()
    )

    # Group by day and calculate the cumulative sum of 'q' for each day
    X_hourly["daily_cumsum"] = X_hourly.groupby(X_hourly["t_1h"].dt.to_period("d"))[
        "q"
    ].cumsum()

    return X_hourly

## Merging of all data

In [26]:
def data_engineered(
    df_train, df_test, weather, underground_transport, overground_transport, cars_count
):
    """
    Merging the data
    """

    def merging_data(data, weather, public_transport, car_traffic):
        data = data.merge(weather, on="date", how="left")
        data = data.merge(
            public_transport[["year", "month", "day", "NB_VALD"]],
            on=["year", "month", "day"],
            how="left",
        )
        data = data.merge(car_traffic, left_on="date", right_on="t_1h", how="left")

        return data

    train_min, train_max = df_train["date"].min(), df_train["date"].max()
    test_min, test_max = df_test["date"].min(), df_test["date"].max()

    # Encoding the date
    df_train, df_test = date_encoder(df_train), date_encoder(df_test)

    # Processing the data
    df_train, df_test = train_test_processing(df_train, df_test)

    # Processing weather data
    weather_processed = weather_processing(
        weather, train_min, train_max, test_min, test_max
    )

    # Processing transport data
    transport_processed = transport_processing(
        underground_transport, overground_transport
    )

    # Processing car traffic data
    car_traffic_processed = car_traffic_processing(cars_count, train_min, test_max)

    # Merging the data
    df_train = merging_data(
        df_train, weather_processed, transport_processed, car_traffic_processed
    )

    df_test = merging_data(
        df_test, weather_processed, transport_processed, car_traffic_processed
    )

    return df_train, df_test

# Pipeline creation

In [27]:
X_train, X_test = data_engineered(
    train, test, weather, underground_transport, overground_transport, cars_count
)

# X_train = X_train[X_train["date"] >= "2021-05-01"].dropna()
X_train = X_train.dropna()
y_train = X_train["log_bike_count"]

In [31]:
preprocessor = TableVectorizer()
# model = xgb.XGBRegressor()
# model = RandomForestRegressor()
model = ExtraTreesRegressor()
pipeline = Pipeline([("preprocessor", preprocessor), ("model", model)])

# Model selection

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit

# cv = TimeSeriesSplit(n_splits=3)

# # When using a scorer in scikit-learn it always needs to be better when smaller, hence the minus sign.
# scores = cross_val_score(
#     pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
# )
# print("RMSE: ", scores)
# print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

# Model finetuning

In [ ]:
# from sklearn.model_selection import TimeSeriesSplit

# cv = TimeSeriesSplit(n_splits=3)

# # When using a scorer in scikit-learn it always needs to be better when smaller, hence the minus sign.
# scores = cross_val_score(
#     pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
# )
# print("RMSE: ", scores)
# print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

# Final model fitting

In [32]:
pipeline.fit(X_train.drop(columns=["bike_count", "log_bike_count"]), y_train)
y_pred = pipeline.predict(X_test)

In [75]:
# import matplotlib.pyplot as plt
# plt.hist(y_train, bins=100)
# plt.show()

In [76]:
# import matplotlib.pyplot as plt
# plt.hist(y_pred, bins=100)
# plt.show()

# Export

In [33]:
pd.DataFrame(y_pred, columns=["log_bike_count"]).reset_index().rename(
    columns={"index": "Id"}
).to_csv("/Users/pierrehaas/bike_counters/predictions.csv", index=False)

In [ ]:
# kaggle competitions submit -c msdb-2024 -f submission.csv -m "Message"